# Part o Speech Tagging of Portugese Sentences

Part of speech (PoS) taggins is a fundamental technique in most natural language recognition projects, in which we try to determine the role each word plays on a sentence. For example, the sentence "Toto, I have a feeling that we are not in Kansas anymore." could be tagged as:

| Word    | Tag         |
|---------|-------------|
| Toto    | Proper noun |
| ,       | Punctuation |
| I       | Pronoun     |
| have    | Verb        |
| a       | Determiner  |
| feeling | Noun        |
| that    | Preposition |
| we      | Pronoun     |
| are     | Verb        |
| not     | Adverb      |
| in      | Preposition |
| Kansas  | Noun        |
| anymore | Adverb      |
| .       | Punctuation |

However, on this notebbok, we will be performing PoS tagging over portuguese sentences, as english PoS tagging has already been covered in many other projects. Therefore, this discussion will be carried out in portuguese from this point onwards.

If you want to learn about pos-tagging in english, you can checkout [this article](https://medium.com/data-science-in-your-pocket/pos-tagging-using-hidden-markov-models-hmm-viterbi-algorithm-in-nlp-mathematics-explained-d43ca89347c4) by Mehul Gupta who does a job that is honestly better than mine on explaining the topic in detail. You can also easily find [many implementations of PoS tagging on GitHub](https://github.com/search?q=pos-tagging).

## Hidden Markov Model

Nesses exemplos, usaremos a implementação do algoritmo _Hidden Markov Model_ (Modelo Oculto de Markov) oferecido pela biblioteca `nltk`. Esse modelo tem diversas aplicações além de _PoS tagging_ em áreas como termodinâmica, estatística, ecnomia, teoria da informação, entre outras.

Teremos um processo stocastico, mais especificamente um processo Markoviano, ou seja, temos uma sequência de variáveis aleatórias que podem assumir um conjunto de estados $Q$ onde a probabilidade de cada estado depende apenas do estado assumido na variável anterior, chamaeremos essa sequência de $X$. Teremos também uma senquência de valores observáveis $Y$ onde a ocorrência de $y_n$ está relacionada com a ocorrência de $x_n$. Baseado nisso teremos duas tabelas de probabilidade: as probabilidades de transição e as probabilidades de emissão. As probabilidades de transição indicam a probabilidade de $x_n$ assumir um dado estado baseado no estado de $x_{n-1}$:

$$P(x_n|x_{n-1})$$

Enquanto as probabilidades de emissão indicam a probabilidade de um obervável $Y_n$ ser emitido por um estado de $X$:

$$P(y_n|x_n)$$

No nosso caso, os estados assumíveis serão classes de palavras existentes na língua portugesa que aqui serão representados com: 

| Classe de palavra                      | Tag        |
|----------------------------------------|------------|
| Artigo                                 | ART        |
| Adjetivo                               | ADJ        |
| Nome                                   | N          |
| Nome próprio                           | NPROP      |
| Pronome adjetivo                       | PROADJ     |
| Pronome substantivo                    | PROSUB     |
| Pronome pessoal                        | PROPESS    |
| Pronome conectivo subordinado          | PRO-KS     |
| Pronome conectivo subordinado relativo | PRO-KS-REL |
| Advérbio                               | ADV        |
| Advérbio conectivo subordinativo       | ADV-KS     |
| Advérbio relativo subordinativo        | ADV-KS-REL |
| Conjunção coordenativa                 | KC         |
| Conjunção subordinativa                | KS         |
| Preposição                             | PREP       |
| Interjeição                            | IN         |
| Verbo                                  | V          |
| Verbo auxiliar                         | VAUX       |
| Particípio                             | PCP        |
| Palavra denotativa                     | PDEN       |
| Moeda corrente                         | CUR        |

as tags ainda podem ser acompanhadas de informações adicionais denotadas por:

| Informação            | Modificador (_TAG_\|MOD) |
|-----------------------|--------------------------|
| Estrangeirismo        | EST                      |
| Apostos               | AP                       |
| Dados                 | DAD                      |
| Números de telefone   | TEL                      |
| Datas                 | DAT                      |
| Horas                 | HOR                      |
| Contrações e ênclises | +                        |
| Mesóclise             | !                        |


A sequência $X$ será a sequência de tags atribuídas a uma sentença fornecida e a sequência $Y$ será composta das palavras que formam a sentença. Ou seja, teremos que as probabilidades de transição serão probabilidades de que uma classe de palavra seja precedida por outra, enquanto as probabilidades de emissão serão as probabilidades de que uma determinada palavra tenha o papel de uma determinada classe de discurso. Por exemplo, suponha que a tenhamos apenas três classes de palavras: nomes, verbos e advérbios, uma possível tabela de probabilidades de transição seria:

|              | **Nome** | **Verbo** | **Advérbio** |
|--------------|----------|-----------|--------------|
| **Nome**     | 0.6      | 0.1       | 0.3          |
| **Verbo**    | 0.5      | 0.1       | 0.4          |
| **Advérbio** | 0.5      | 0.4       | 0.1          |

caso o nosso vocabulário possua apenas as palavras armando, faz e rapidamente, uma possível tabela de probabilidades de emissão seria:

|                 | **Nome** | **Verbo** | **Advérbio** |
|-----------------|----------|-----------|--------------|
| **Armando**     | 0.5      | 0.5       | 0.0          |
| **Faz**         | 0.1      | 0.9       | 0.0          |
| **Rapidamente** | 0.0      | 0.0       | 1.0          |

O _Hidden Markov Model_ monta uma máquina de estados e trabalha com iterações baseadas nas duas probabilidades mencionadas para convergir e determinar uma classificação adequada para as palavras de uma sentença. Veremos agora o exemplo utilizando a implementação da `nltk` do algoritmo.

## Implementação

In [18]:
import numpy as np
import nltk
from nltk.tag.hmm import HiddenMarkovModelTrainer
from random import randint
from pandas import DataFrame

Utilizaremos o corpus disponível [nesse link](http://nilc.icmc.usp.br/macmorpho/macmorpho-v3.tgz) cuja documentação pode ser obtida [aqui](http://nilc.icmc.usp.br/macmorpho/macmorpho-manual.pdf).

In [50]:
corpus_train_path = './macmorpho-train.txt'
corpus_test_path = './macmorpho-test.txt'

A função `read_tags` recebe o caminho para o arquivo e realiza o pré processamento do texto. Em primeiro lugar, colocamos todo o texto em letras minúsculas para evitar problemas. Em seguida observamos o formato do corpus que é uma sequência de tokens `PALAVRA_TAG` separados por espaço ou quebra de linha, então substituímos todos as quebras de linha por espaços e fazemos um split nesses espaços para separar esses tokens. Então fazemos um split por "_" para criar tuplas `(PALAVRA, TAG)`. Por fim, separamos o texto em sentenças assumindo que sentenças terminam em ".", "!" ou "?".

A função `read_sentence_text` é um helper que converte um vetor de tuplas na frase original.

In [6]:
def read_tags(corpus_path: str):
    corpus_file = open(corpus_path, encoding='utf-8')
    corpus = corpus_file.read().replace('\n', ' ').lower()
    words = [tuple(word_tag.split('_')) for word_tag in corpus.split(' ')]
    
    sentences = [[]]
    sentence_terminators = ['.', '!', '?']
    for word in words:
        sentences[-1].append(word)
        if word[0] in sentence_terminators:
            sentences.append([])
    sentences.pop()        
    
    return sentences

def read_sentence_text(sentence):
    return ' '.join([word[0] for word in sentence])

Então pre-processamos nosso corpus e treinamos o modelo com a classe `HiddenMarkovModelTrainer` do módulo `hmm` da `nltk`.

In [9]:
sentences = read_tags(corpus_train_path)
trainer = HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(sentences[:34724])

Um exemplo da nossa organização dos dados é apresentado abaixo:

In [22]:
sentence_index = randint(0, len(sentences))
print("Frase:")
print(read_sentence_text(sentences[sentence_index]))
print("Representação:")
DataFrame(sentences[sentence_index])

Frase:
em julho , esse segmento já havia registrado queda de 4,78 % nos preços .
Representação:


,0,1
0,em,prep
1,julho,n
2,",",pu
3,esse,proadj
4,segmento,n
5,já,adv
6,havia,v
7,registrado,pcp
8,queda,n
9,de,prep


Com o modelo treinado, podemos realizar amostras de classificações para palavras com as funções abaixo.

In [27]:
def tag_word(word: str):
  return tagger.tag([word.lower()])[0][1]

def tag_sentence(sentence: str):
  return tagger.tag_sents([sentence.split(' ')])[0]

def print_tag(word: str):
  print(f'{word}:', tag_word(word))

Palavras:

In [38]:
print_tag('fugiu')
print_tag('escritório')
print_tag('rapidamente')
print_tag('R$')
print_tag('armando')
print_tag('seu')
print_tag('feliz')

fugiu: v
escritório: n
rapidamente: adv
R$: cur
armando: nprop
seu: proadj
feliz: adj
credo: n


Sentenças:

In [47]:
tag_sentence('mesmo estando atrasado , parou para cumprimentá-lo')

[('mesmo', 'pden'),
 ('estando', 'v'),
 ('atrasado', 'pcp'),
 (',', 'pu'),
 ('parou', 'v'),
 ('para', 'prep'),
 ('cumprimentá-lo', 'n')]

In [49]:
tag_sentence('o galo ganhou mais uma vez , ai credo')

[('o', 'art'),
 ('galo', 'n'),
 ('ganhou', 'v'),
 ('mais', 'adv'),
 ('uma', 'art'),
 ('vez', 'n'),
 (',', 'pu'),
 ('ai', 'in'),
 ('credo', 'n')]

In [48]:
tag_sentence('que a força esteja com você')

[('que', 'ks'),
 ('a', 'art'),
 ('força', 'n'),
 ('esteja', 'v'),
 ('com', 'prep'),
 ('você', 'propess')]

## Testes do modelo

Agora vamos utilizar o conjunto de dados de teste incluído junto ao corpus para testar a qualidade do modelo treinado.

In [ ]:
test_sentences = read_tags(corpus_test_path)